In [1]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 45.4 MB/s eta 0:00:00


In [2]:
# Import libraries
import librosa
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('punkt')

# 1. Audio Preprocessing
def audio_to_text(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return text
        except Exception as e:
            print(f"Error: {e}")
            return ""

# 2. Feature Extraction Functions
def extract_acoustic_features(audio_path):
    y, sr = librosa.load(audio_path)

    # Speech rate (syllables per second approximation)
    duration = librosa.get_duration(y=y)
    words = len(word_tokenize(audio_to_text(audio_path)))
    speech_rate = words/(duration/60)  # Words per minute

    # Pitch features
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitch_mean = np.mean(pitches[pitches > 0])
    pitch_std = np.std(pitches[pitches > 0])

    # Pause analysis
    intervals = librosa.effects.split(y, top_db=20)
    pause_durations = [librosa.samples_to_time(i[1]-i[0], sr=sr) for i in intervals]

    return {
        'speech_rate': speech_rate,
        'pitch_mean': pitch_mean,
        'pitch_std': pitch_std,
        'avg_pause_duration': np.mean(pause_durations),
        'pause_frequency': len(pause_durations)
    }

def linguistic_features(text):
    hesitation_count = text.lower().count('uh') + text.lower().count('um')
    words = word_tokenize(text)
    word_count = len(words)

    # Sentence complexity (simple metric)
    sentences = nltk.sent_tokenize(text)
    avg_sentence_length = sum(len(s.split()) for s in sentences)/len(sentences) if sentences else 0

    return {
        'hesitation_count': hesitation_count,
        'word_count': word_count,
        'avg_sentence_length': avg_sentence_length,
        'hesitation_ratio': hesitation_count/word_count if word_count else 0
    }

# 3. Data Processing Pipeline
def process_audio_samples(file_paths):
    features = []

    for path in file_paths:
        text = audio_to_text(path)
        acoustic = extract_acoustic_features(path)
        linguistic = linguistic_features(text)

        combined_features = {**acoustic, **linguistic}
        features.append(combined_features)

    return pd.DataFrame(features)

# 4. Machine Learning Models
def apply_unsupervised_learning(df):
    # Normalize features
    df_normalized = (df - df.mean()) / df.std()

    # Handle NaN values (replace with mean of the column)
    df_normalized = df_normalized.fillna(0) # Replace NaN with the mean of each column

    # Clustering
    # Check if there are enough samples for clustering
    if df_normalized.shape[0] >= 2:  # Check if there are at least 2 samples
        kmeans = KMeans(n_clusters=2, random_state=42)
        df['cluster'] = kmeans.fit_predict(df_normalized)
    else:
        # If not enough samples, assign all to a single cluster or handle appropriately
        df['cluster'] = 0  # Assign all to cluster 0, for example

    # Anomaly Detection
    iso_forest = IsolationForest(contamination=0.1)
    df['anomaly_score'] = iso_forest.fit_predict(df_normalized)

    # Similarity Scoring
    reference = df_normalized.mean().values.reshape(1, -1)
    df['similarity_score'] = cosine_similarity(df_normalized, reference)

    return df

# 5. Usage Example
if __name__ == "__main__":
    # Simulated file paths (replace with actual paths)
    audio_files = ['/content/drive/MyDrive/most stressful 10 minutes of my existence (10 minute song) [peG6ZC1M-Uw] (1)-[AudioTrimmer.com].wav']

    # Process samples
    feature_df = process_audio_samples(audio_files)

    # Apply ML
    results_df = apply_unsupervised_learning(feature_df)

    # Display results
    print("Analysis Results:")
    print(results_df[['cluster', 'anomaly_score', 'similarity_score']])

    # Save results
    results_df.to_csv('cognitive_analysis_results.csv', index=False)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Analysis Results:
   cluster  anomaly_score  similarity_score
0        0              1               0.0
